# Maschinelles Lernen
## Übung Termin 1 -  Datenaufbereitung
### Aufbau der Übung

**1. Importieren von Daten**
- Datensätze einladen  
- Zusammenführen und Filtern von Datensätzen

**2. Datenaufbereitung**
- Ersetzen von Werten unterhalb der Nachweisgrenze  
- Ersetzen von kategorischen und numerischen *NaN*-Einträgen  
- Feature-Aggregation  
- Aufteilen in Trainings- und Test-Split  
- Skalieren (Normieren/Standardisieren)  
- Encodieren (Binär / One-Hot / Target)  
- Speichern der vorbereiteten Daten

**3. Übungsaufgabe**
- Eigenständige Anwendung der Schritte


#### Import der Python Bibliotheken


In [1]:
# Basic packages
import pandas as pd       # Datenmanipulation und -analyse (Tabellenstrukturen)
import numpy as np        # Numerische Operationen, Arrays, Zufallsfunktionen
import matplotlib.pyplot as plt  # Erstellung von Diagrammen und Visualisierungen

---
---

# 1. Importieren der Daten.
## 1.1 Datensätze einladen
Wir verwenden zwei Datensätze
- Nitratdatensatz (Nitratmessungen an GW-Messstellen)
- [Corine Landnutzungarten](https://land.copernicus.eu/pan-european/corine-land-cover/clc2018) an den GW-Messstellen


Wir verwenden die Funktion <span style="color:blue">**pandas.read_csv**</span> &rarr; [Hilfe](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) um die Dateien zu importieren.

### 1.1.1 Nitratdatensatz:

In [2]:
# Nitratdatensatz einlesen
data_NO3 = pd.read_csv('Nitratmessungen.csv', sep=',', encoding="utf-8", index_col=0)

# Zeige die ersten 5 Zeilen des Datensatzes an
data_NO3.head(5)

,GW-Nummer,Messstelle,Datum,NO3 [mg/l],pH [ -],O2 [mg/l],O2-Sätt [%],T [°C],RECHTSWERT,HOCHWERT,HYDROGEOLE,AQUIFER,HYDROGEOL3
0,918/069-2,"BBR Betonwerk, Umkirch",11.10.2006 15:20,"9,4","6,45","2,4","23,3","14,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)
1,918/069-2,"BBR Betonwerk, Umkirch",17.09.2007 14:20,12,"6,39","2,1","25,0","21,8",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)
2,918/069-2,"BBR Betonwerk, Umkirch",23.09.2008 08:05,"7,6","6,58","2,5","23,0",NaN,3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)
3,918/069-2,"BBR Betonwerk, Umkirch",10.11.2008 09:10,"12,2","6,54","1,2","12,0","15,4",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)
4,918/069-2,"BBR Betonwerk, Umkirch",09.09.2009 14:08,12,"6,46","1,3","14,0","20,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)


In [3]:
# Ausgabe der Datentypen aller Spalten im DataFrame
# (z.B. int64, float64, bool, datetime64, timedelta, object usw.)
data_NO3.dtypes

GW-Nummer       object
Messstelle      object
Datum           object
NO3 [mg/l]      object
pH [ -]         object
O2 [mg/l]       object
O2-Sätt [%]     object
T [°C]          object
RECHTSWERT       int64
HOCHWERT         int64
HYDROGEOLE     float64
AQUIFER         object
HYDROGEOL3      object
dtype: object

<div style="background-color: #fff8dc; border-left: 6px solid #f0ad4e; padding: 10px; margin: 10px 0;">
<b>Hinweis:</b><br><br>
Beim Überprüfen der Datentypen fällt auf, dass mehrere Spalten einen falschen Typ besitzen:
<ul>
<li>Die Spalte <code>Datum</code> liegt aktuell als <code>object</code>-Typ vor, sollte jedoch in das Datetime-Format (<code>datetime64</code>) umgewandelt werden.</li>
<li>Die Spalten mit physikochemischen Eigenschaften sowie der Nitratgehalt sollten in <code>float</code>-Werte konvertiert werden, um numerische Berechnungen korrekt durchführen zu können.</li>
</ul>
Diese notwendigen Anpassungen werden wir im Abschnitt <b>Datenaufbereitung</b> vornehmen.
</div>



### 1.1.2 Corine Landnutzung für Messstellen:
Die **CORINE Landnutzungsdaten** der Europäischen Umweltagentur klassifizieren die Landnutzung in Europa.  
Sie umfassen Kategorien wie:
- **Ackerland** (Code 1)
- **Wälder** (Code 2)
- **Siedlungsgebiete** (Code 3)
- sowie weitere Landnutzungsformen.

Die Daten werden in Form von **Raster- oder Vektordaten** bereitgestellt, häufig in einem **rasterbasierten Format** mit spezifischer räumlicher Auflösung.

In der Datei **`Corinedaten.txt`** wurden in einem vorbereitenden Schritt die Landnutzungsklassen direkt an den Standorten der Messstellen erfasst.

Weitere Informationen zu den verfügbaren **Bändern und Klassen der CORINE-Daten** findet ihr hier:  
&rarr; [CORINE Land Cover Datenbeschreibung (Google Earth Engine)](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_CORINE_V20_100m#bands)

In [4]:
# Corine Datensatz einlesen
data_Corine = pd.read_csv('Corinedaten.txt', sep=';',decimal=',',  encoding='utf-8', index_col=0)

# Zeige die ersten 5 Zeilen des Datensatzes an
data_Corine.head(5)

,GW_NUMMER,NAME,RECHTSWERT,HOCHWERT,GEMEINDE_K,AQUIFER_KZ,EINZUGSGEB,HYDROGEOLE,HYDROGEOL2,MESSNETZ1_,MESSNETZ2_,GEMEINDE,AQUIFER,EINZUGSGE2,HYDROGEOL3,MESSNETZ1,MESSNETZ2,RASTERVALU
FID,,,,,,,,,,,,,,,,,,
0,1/022-9,"QSS HERTINGERQU.1+2, KANDERN",3395475.0,5288420.0,8336104.0,31.0,2.333220e+09,11.0,2.0,,,MALSBURG-MARZELL,MALM WEIßJURA (SCHWÄBISCHE ALB),Engebach,Oberjura (Raurasische Fazies) (GWL),#800000000010,#800000000002,311
1,1/117-3,"QF KREBSBRUNNENQ., ETTENHEIM",3419400.0,5345550.0,8317026.0,81.0,2.338940e+09,45.0,69.0,,,"ETTENHEIM, STADT",BUNTSANDSTEIN,Ettenbach,Unterer und Mittlerer Buntsandstein (GWL),#800000000010,#800000000002,311
2,1/119-9,"TB SCHLAGBR.4, VOERSTETTEN",3414860.0,5325450.0,8316045.0,4.0,2.338892e+09,3.0,8.0,,,VÖRSTETTEN,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Schobbach,Quartäre Kiese und Sande (GWL),#800000000012,,211
3,1/120-8,"QF2 KLEISLEWALDQ., ZASTLER",3423200.0,5309100.0,8315084.0,91.0,2.338830e+09,50.0,18.0,,,OBERRIED,KRISTALLIN (SCHWARZWALD),Dreisam unterh. Wagensteigbach oberh. Brugga,"Paläozoikum, Kristallin (GWG)",#800000000010,#800000000002,312
4,1/121-0,"Q STOLLENQUELLE HINTERES ELEND, MÜNSTERTAL",3413785.0,5303338.0,8315130.0,91.0,2.336410e+09,50.0,18.0,,,MÜNSTERTAL/ SCHWARZWALD,KRISTALLIN (SCHWARZWALD),Neumagen oberhalb von Talbach,"Paläozoikum, Kristallin (GWG)",#800000000010,#800000000002,231


## 1.2. Zusammenführen und Filtern der Datensätze
### Zusammenführen der Datensätze

Zusammenführen der beiden Datensätze mithilfe der <span style="color:blue">**`.merge`**</span>-Funktion für Pandas-DataFrames.  
&rarr; [Hilfe zur `.merge`-Funktion (pandas-Dokumentation)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)

- Die **gemeinsame Spalte** für das Zusammenführen ist die **`Grundwassernummer`**.
- Vom zweiten Datensatz benötigen wir lediglich die Spalte **`RASTERVALU`**, die die Landnutzungsdaten aus der CORINE Land Cover enthält.

Weitere Informationen zu den verfügbaren **Bändern und Landnutzungsklassen** finden Sie hier:  
&rarr; [CORINE Land Cover Datenbeschreibung (Google Earth Engine)](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_CORINE_V20_100m#bands)


### 1.2.1 Zusammenführen der Datensätze
<div style="background-color: #e7f3fe; border-left: 6px solid #2196F3; padding: 10px; margin: 10px 0;">
<b>ℹ️ Info:</b><br><br>
Für die weitere Verarbeitung benötigen wir aus den Corine-Daten nur die Spalte <code>RASTERVALU</code> sowie die <code>GW-Nummer</code> als Schlüssel, um beim Zusammenführen die richtigen Werte zuzuweisen.
</div>


In [5]:
# Führe die beiden Datensätze anhand der gemeinsamen Spalte "GW-Nummer" zusammen und behalte nur die benötigten Spalten
joined_data = data_NO3.merge(data_Corine[['GW_NUMMER','RASTERVALU']], how='inner', left_on='GW-Nummer', right_on='GW_NUMMER')

# Zeige die ersten 5 Zeilen des zusammengeführten Datensatzes an
joined_data.head(5)

,GW-Nummer,Messstelle,Datum,NO3 [mg/l],pH [ -],O2 [mg/l],O2-Sätt [%],T [°C],RECHTSWERT,HOCHWERT,HYDROGEOLE,AQUIFER,HYDROGEOL3,GW_NUMMER,RASTERVALU
0,918/069-2,"BBR Betonwerk, Umkirch",11.10.2006 15:20,"9,4","6,45","2,4","23,3","14,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
1,918/069-2,"BBR Betonwerk, Umkirch",17.09.2007 14:20,12,"6,39","2,1","25,0","21,8",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
2,918/069-2,"BBR Betonwerk, Umkirch",23.09.2008 08:05,"7,6","6,58","2,5","23,0",NaN,3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
3,918/069-2,"BBR Betonwerk, Umkirch",10.11.2008 09:10,"12,2","6,54","1,2","12,0","15,4",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
4,918/069-2,"BBR Betonwerk, Umkirch",09.09.2009 14:08,12,"6,46","1,3","14,0","20,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121


In [6]:
# Zusammenführen der beiden Datensätze anhand der gemeinsamen Spalte "GW-Nummer"
# - Übernehme nur die Spalten "GW_NUMMER" und "RASTERVALU" aus dem Corine-Datensatz
# - Verwende einen inneren Join (nur übereinstimmende Einträge bleiben erhalten)
joined_data = data_NO3.merge(
    data_Corine[['GW_NUMMER', 'RASTERVALU']], 
    how='inner', 
    left_on='GW-Nummer', 
    right_on='GW_NUMMER'
)

# Entferne die doppelt vorhandene Schlüsselspalte
#joined_data = joined_data.drop(columns=["GW_NUMMER"])

# Vorschau: Zeige die ersten 5 Zeilen des zusammengeführten Datensatzes
joined_data.head(5)


,GW-Nummer,Messstelle,Datum,NO3 [mg/l],pH [ -],O2 [mg/l],O2-Sätt [%],T [°C],RECHTSWERT,HOCHWERT,HYDROGEOLE,AQUIFER,HYDROGEOL3,GW_NUMMER,RASTERVALU
0,918/069-2,"BBR Betonwerk, Umkirch",11.10.2006 15:20,"9,4","6,45","2,4","23,3","14,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
1,918/069-2,"BBR Betonwerk, Umkirch",17.09.2007 14:20,12,"6,39","2,1","25,0","21,8",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
2,918/069-2,"BBR Betonwerk, Umkirch",23.09.2008 08:05,"7,6","6,58","2,5","23,0",NaN,3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
3,918/069-2,"BBR Betonwerk, Umkirch",10.11.2008 09:10,"12,2","6,54","1,2","12,0","15,4",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
4,918/069-2,"BBR Betonwerk, Umkirch",09.09.2009 14:08,12,"6,46","1,3","14,0","20,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121


In [7]:
# Gebe Datentypen für die einzelnen Spalten des zusammengeführten Dataframes aus
joined_data.dtypes

GW-Nummer       object
Messstelle      object
Datum           object
NO3 [mg/l]      object
pH [ -]         object
O2 [mg/l]       object
O2-Sätt [%]     object
T [°C]          object
RECHTSWERT       int64
HOCHWERT         int64
HYDROGEOLE     float64
AQUIFER         object
HYDROGEOL3      object
GW_NUMMER       object
RASTERVALU       int64
dtype: object

### 1.2.2 Filtern der Datensätze
### Auswahl relevanter Spalten für die weitere Verarbeitung

Für die weitere Verarbeitung benötigen wir nur noch die folgenden Spalten:

- **Messstelle**
- **GW-Nummer**
- **Datum**
- **NO3** (Nitratgehalt)
- **O2** (Sauerstoffgehalt)
- **RASTERVALU** (Corine Land Cover Klassifikation)
- **Hydrogeologie**

In [8]:
# Erstelle DataFrame mit ausgewählten Spalten (columns) und konvertiere 'Datum' in ein Datetime-Objekt
selected_columns = ['Messstelle', 'GW-Nummer', 'Datum', 'NO3 [mg/l]', 'O2 [mg/l]', 'RASTERVALU', 'HYDROGEOL3']
data = joined_data[selected_columns].copy()  # selected_columns auswählen und eine Kopie erstelle

# 'Datum' in ein Datetime-Objekt konvertieren
data['Datum'] = pd.to_datetime(data['Datum'], format='%d.%m.%Y %H:%M')  

# Zeige die ersten 5 Zeilen des zusammengeführten Datensatzes an
data.head(5)
data.dtypes

Messstelle            object
GW-Nummer             object
Datum         datetime64[ns]
NO3 [mg/l]            object
O2 [mg/l]             object
RASTERVALU             int64
HYDROGEOL3            object
dtype: object

In [9]:
# Erstellen eines DataFrames mit den ausgewählten Spalten
# - Auswahl der wichtigsten Merkmale für die weitere Analyse
selected_columns = [
    'Messstelle', 
    'GW-Nummer', 
    'Datum', 
    'NO3 [mg/l]', 
    'O2 [mg/l]', 
    'RASTERVALU', 
    'HYDROGEOL3'
]

# Erstellen einer Kopie mit den ausgewählten Spalten
data = joined_data[selected_columns].copy()
# Alternative mit loc: data = joined_data.loc[:, selected_columns].copy()

# Konvertieren der Spalte 'Datum' in ein Datetime-Objekt
# - Annahme: Datumsformat ist Tag.Monat.Jahr Stunde:Minute (z.B. 24.12.2020 13:45)
data['Datum'] = pd.to_datetime(data['Datum'], format='%d.%m.%Y %H:%M')

# Vorschau: Zeige die ersten 5 Zeilen des neu erstellten DataFrames
display(data.head(5))

# Kontrolle: Zeige die Datentypen der Spalten an
data.dtypes


,Messstelle,GW-Nummer,Datum,NO3 [mg/l],O2 [mg/l],RASTERVALU,HYDROGEOL3
0,"BBR Betonwerk, Umkirch",918/069-2,2006-10-11 15:20:00,"9,4","2,4",121,Quartäre Kiese und Sande (GWL)
1,"BBR Betonwerk, Umkirch",918/069-2,2007-09-17 14:20:00,12,"2,1",121,Quartäre Kiese und Sande (GWL)
2,"BBR Betonwerk, Umkirch",918/069-2,2008-09-23 08:05:00,"7,6","2,5",121,Quartäre Kiese und Sande (GWL)
3,"BBR Betonwerk, Umkirch",918/069-2,2008-11-10 09:10:00,"12,2","1,2",121,Quartäre Kiese und Sande (GWL)
4,"BBR Betonwerk, Umkirch",918/069-2,2009-09-09 14:08:00,12,"1,3",121,Quartäre Kiese und Sande (GWL)


Messstelle            object
GW-Nummer             object
Datum         datetime64[ns]
NO3 [mg/l]            object
O2 [mg/l]             object
RASTERVALU             int64
HYDROGEOL3            object
dtype: object

<div style="background-color: #fff3cd; border-left: 6px solid #ffa502; padding: 10px; margin: 10px 0;">
<b>Vorsicht:</b><br><br>
Normalerweise könnten wir die Werte einfach mit <code>.astype(float)</code> umwandeln.  
Wenn ihr das versucht, werdet ihr jedoch eine Fehlermeldung erhalten.  
Der Grund: Die Zahlen verwenden ein **Komma** anstelle eines **Punktes** als Dezimaltrennzeichen.
<br><br>
Daher müssen wir vor der Umwandlung das Komma durch einen Punkt ersetzen.<br><br>
Es lohnt sich, die betroffenen Spalten genauer zu betrachten, bevor wir die Konvertierung durchführen.
</div>


---
---

# 2. Datenaufbereitung

<div style="background-color: #e7f3fe; border-left: 6px solid #2196F3; padding: 10px; margin: 10px 0;">
<b>Hinweis:</b><br><br>
Bevor ihr Manipulationen an einem DataFrame durchführt, kann es hilfreich sein, eine <b>Kopie</b> des DataFrames zu erstellen,  
um mögliche Fehlermeldungen zu vermeiden – insbesondere wenn ihr Zellen mehrmals ausführt.<br><br>
Dies könnt ihr mit folgendem Befehl tun:
<pre><code>df = data.copy()</code></pre>
Auf diese Weise müsst ihr nicht das gesamte Skript erneut ausführen, sondern könnt einfach ab dieser Zelle weiterarbeiten.
</div>


In [10]:
# Erstelle eine Kopie des DataFrame
df= data.copy()

Spalte `"NO3 [mg/l]"`

In [11]:
# Sortiere die Werte der Spalte "NO3 [mg/l]" in aufsteigender Reihenfolge
# Hinweis: Der Datensatz enthält Werte unterhalb der Nachweisgrenze (NWG)
df["NO3 [mg/l]"].sort_values()

5026      0,1
1707      0,1
5873      0,1
5543      0,2
6549      0,2
        ...  
8294    < 0,5
1366    < 0,5
1367    < 0,5
8248    < 0,5
5724    < 0,5
Name: NO3 [mg/l], Length: 12178, dtype: object

> **Vorsicht:** Wir sehen hier, dass es Werte gibt, die unterhalb der Nachweisgrenze liegen. Diese wurden mit "<" gekennzeichnet. Da "Pandas" nicht mit "<" arbeiten kann, müssen wir a) die Zeichen entfernen und b) überlegen, was wir mit den Werten unterhalb der Nachweisgrenze machen.


Spalte `"O2 [mg/l]"`

In [12]:
# Sortiere die Werte der Spalte in aufsteigender Reihenfolge
df["O2 [mg/l]"].sort_values() #--> beeinhaltet NAN Werte

8181     0,1
3490     0,1
5548     0,1
3485     0,1
685      0,1
        ... 
12160    NaN
12161    NaN
12162    NaN
12163    NaN
12164    NaN
Name: O2 [mg/l], Length: 12178, dtype: object

<div style="background-color: #fff3cd; border-left: 6px solid #ffa502; padding: 10px; margin: 10px 0;">
<b>Vorsicht:</b><br><br>
Wir haben festgestellt, dass <b>NaN-Werte</b> im Datensatz vorhanden sind.  
Möglicherweise wurden bestimmte Parameter nicht gemessen.  
<br><br>
In vielen Fällen müssen fehlende Werte geschätzt oder <b>imputiert</b> werden.  
Eine allgemein gültige Regel, wie viele Werte maximal geschätzt werden dürfen, gibt es jedoch nicht.  
Dies hängt unter anderem ab von:
<ul>
<li>der Art der Daten,</li>
<li>der Qualität der vorhandenen Daten,</li>
<li>der gewählten Analysemethode</li>
<li>und dem konkreten Anwendungsfall.</li>
</ul>

<b>Praxisempfehlung:</b>  
Oft wird empfohlen, dass nicht mehr als <b>5–10%</b> der Daten imputiert werden sollten,  
um die Zuverlässigkeit der Ergebnisse zu gewährleisten.
</div>


In [13]:
# FYI: Zähle die Anzahl von "<"-Symbolen und NaN-Werten in ausgewählten Spalten

# Spaltennamen definieren
columns = ['O2 [mg/l]', 'NO3 [mg/l]']

# Schleife durch jede angegebene Spalte
for col in columns:
    # Anzahl der "<"-Zeichen zählen (nur in Strings sinnvoll)
    count_lessthan = df[col].astype(str).str.count('<').sum()
    
    # Anzahl der NaN-Werte zählen
    count_nan = df[col].isna().sum()
    
    # Gesamtzahl der Einträge (inklusive NaN)
    total_entries = len(df)
    
    # Prozentsätze berechnen
    percent_lessthan = round((count_lessthan / total_entries) * 100, 2)
    percent_nan = round((count_nan / total_entries) * 100, 2)
    
    # Strukturierte Ausgabe
    print(f"🔹 Spalte: '{col}'")
    print(f"   - '<'-Zeichen: {count_lessthan} Einträge ({percent_lessthan}%)")
    print(f"   - NaN-Werte: {count_nan} Einträge ({percent_nan}%)\n")


🔹 Spalte: 'O2 [mg/l]'
   - '<'-Zeichen: 1413 Einträge (11.6%)
   - NaN-Werte: 475 Einträge (3.9%)

🔹 Spalte: 'NO3 [mg/l]'
   - '<'-Zeichen: 2110 Einträge (17.33%)
   - NaN-Werte: 0 Einträge (0.0%)



### 2.1 Ersetzen von Werten unterhalb der Nachweisgrenze (NWG)

Die **Nachweisgrenze (NWG)** ist der kleinste messbare Wert eines Analyten in einem bestimmten System oder Labor.  
Werte unterhalb der NWG gelten als **nicht signifikant**.

In diesem Schritt werden solche Werte durch **`0,5 × NWG`** ersetzt,  
um eine grobe Schätzung des tatsächlichen Wertes zu erhalten – basierend auf der Annahme,  
dass der wahre Wert irgendwo zwischen 0 und der Nachweisgrenze liegt.

<div style="background-color: #fff3cd; border-left: 6px solid #ffa502; padding: 10px; margin-top: 15px;">
<b> Hinweis:</b><br><br>
Die Behandlung von Werten unterhalb der NWG ist nicht in allen Fällen sinnvoll.  
Je nach Datentyp, Analysemethode und wissenschaftlichem Kontext kann es geeigneter sein,  
fehlende Werte gezielt auszuschließen oder alternative Schätzverfahren einzusetzen.
</div>

In [14]:
# --- Codeabschnitt: Funktion zum Ersetzen von Werten unterhalb der NWG ---
def ersetze_unter_NWG(wert):
    """
    Ersetzt Werte unterhalb der Nachweisgrenze (NWG) durch 0.5 * NWG.
    Args:
        wert (int, float oder str): Der Eingabewert (kann numerisch oder als String vorliegen).
    Returns:
        float: 
        - Originalwert, falls bereits numerisch und gültig,
        - oder 0.5 * NWG-Wert, falls Bereichsangabe mit '<' vorliegt.
    """
    # Falls der Wert ein String ist (z.B. "<1,0"), Umwandlung notwendig
    if isinstance(wert, str):
        # Prüfen, ob ein Bereichszeichen "<" enthalten ist
        if '<' in wert:
            # Extrahiere den Zahlenwert, ersetze Komma durch Punkt und halbiere ihn
            bereichswert = float(wert.split('<')[1].replace(',', '.')) / 2
        else:
            # Falls kein "<" enthalten ist, normal in float umwandeln
            bereichswert = float(wert.replace(',', '.'))
    else:
        # Falls der Wert bereits numerisch ist, direkt übernehmen
        bereichswert = wert

    # Rückgabe des bereinigten Wertes
    return bereichswert


### Anwenden der Funktion <span style="color:green"><b>ersetze_unter_NWG</b></span> auf die Messdaten

Wir wenden die <span style="color:green"><b>ersetze_unter_NWG</b></span>-Funktion  
auf die Spalten `'NO3 [mg/l]'` und `'O2 [mg/l]'` an,  
um daraus die neuen Spalten **`NO3`** und **`O2`** zu erstellen.

Verwendete Funktionen:
- <span style="color:blue"><b>.apply</b></span>-Methode &rarr; [Hilfe zur `.apply`-Methode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)
- <span style="color:blue"><b>.drop</b></span>-Methode &rarr; [Hilfe zur `.drop`-Methode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)

Anschließend werden die ursprünglichen Spalten (`'NO3 [mg/l]'` und `'O2 [mg/l]'`) entfernt,  
um den DataFrame übersichtlicher zu gestalten.


In [15]:
# Erstellen neuer Spalten "O2" und "NO3" durch Anwendung der ersetze_unter_NWG-Funktion

# Wandle die Spalte "O2 [mg/l]" um und speichere das Ergebnis in einer neuen Spalte "O2"
df['O2'] = df['O2 [mg/l]'].apply(ersetze_unter_NWG)

# Wandle die Spalte "NO3 [mg/l]" um und speichere das Ergebnis in einer neuen Spalte "NO3"
df['NO3'] = df['NO3 [mg/l]'].apply(ersetze_unter_NWG)

# Löschen der alten Spalten "O2 [mg/l]" und "NO3 [mg/l]", um den DataFrame aufzuräumen
df.drop(columns=['O2 [mg/l]', 'NO3 [mg/l]'], inplace=True)

# Vorschau: Zeige die ersten fünf Zeilen des aktualisierten DataFrames
df.head()


,Messstelle,GW-Nummer,Datum,RASTERVALU,HYDROGEOL3,O2,NO3
0,"BBR Betonwerk, Umkirch",918/069-2,2006-10-11 15:20:00,121,Quartäre Kiese und Sande (GWL),2.4,9.4
1,"BBR Betonwerk, Umkirch",918/069-2,2007-09-17 14:20:00,121,Quartäre Kiese und Sande (GWL),2.1,12.0
2,"BBR Betonwerk, Umkirch",918/069-2,2008-09-23 08:05:00,121,Quartäre Kiese und Sande (GWL),2.5,7.6
3,"BBR Betonwerk, Umkirch",918/069-2,2008-11-10 09:10:00,121,Quartäre Kiese und Sande (GWL),1.2,12.2
4,"BBR Betonwerk, Umkirch",918/069-2,2009-09-09 14:08:00,121,Quartäre Kiese und Sande (GWL),1.3,12.0


### 2.2 Ersetzen von fehlenden Werten (NULL-Values)

Datensätze sind häufig unvollständig und enthalten fehlende Werte, sogenannte **`NULL`-Werte** oder **`NaN`**-Einträge.  
Diese können verschiedene Ursachen haben, etwa unvollständige Erfassungen, Sensorausfälle oder fehlerhafte Datenübertragungen.

Fehlende Werte können problematisch sein, da sie viele statistische Metriken und Machine-Learning-Algorithmen beeinträchtigen – insbesondere wenn diese nicht für fehlende Werte ausgelegt sind.

<div style="background-color: #e7f3fe; border-left: 6px solid #2196F3; padding: 10px; margin-top: 15px;">
<b>Wichtig:</b><br><br>
Es ist entscheidend, den Unterschied zwischen <code>0</code> und <code>NULL</code> zu verstehen:<br>
- <code>0</code> steht für eine tatsächlich gemessene Null.<br>
- <code>NULL</code> bedeutet, dass überhaupt kein Messwert vorhanden ist.
</div>

---

Das vollständige Löschen von Zeilen mit `NULL`-Werten ist nur bei einem sehr geringen Anteil sinnvoll, da ansonsten wertvolle Informationen verloren gehen können.  
Besser ist es in den meisten Fällen, fehlende Werte durch **Imputation** zu ersetzen.

Es gibt verschiedene Imputationsmethoden, beispielsweise:
- Auffüllen mit dem **Mittelwert**, **Median** oder **Mode** der vorhandenen Werte
- Verwendung von **Machine-Learning-Modellen** zur Vorhersage der fehlenden Werte basierend auf anderen verfügbaren Daten.

<div style="background-color: #fff3cd; border-left: 6px solid #ffa502; padding: 10px; margin-top: 15px;">
<b>Hinweis:</b><br><br>
Die Auswahl der geeigneten Imputationsmethode sollte stets sorgfältig erfolgen und validiert werden,  
um Verzerrungen und Fehlinterpretationen in den Analyseergebnissen zu vermeiden.
</div>


#### Typische Methoden zur Kontrolle fehlender Werte

- **`isnull().sum()`**  
  → Zählt die Anzahl der fehlenden Werte (`NaN`) pro Spalte.

- **`isnull().mean()`**  
  → Berechnet den Anteil fehlender Werte pro Spalte (in Prozent, wenn mit 100 multipliziert).


In [16]:
# Kontrolle: Ausgabe der Anzahl fehlender Werte pro Spalte
df.isnull().sum()


Messstelle      0
GW-Nummer       0
Datum           0
RASTERVALU      0
HYDROGEOL3     56
O2            475
NO3             0
dtype: int64

### 2.2.1 Ersetzen von NaN-Werten in numerischen Spalten

Für die numerische Spalte **`O2`** werden die fehlenden Werte durch den Mittelwert ersetzt.

Dies geschieht mithilfe des <span style="color:blue"><b>`fillna()`</b></span>-Befehls für Pandas-DataFrames.


<div style="background-color: #e7f3fe; border-left: 6px solid #2196F3; padding: 10px; margin-top: 15px;">
<b>Zur Info:</b><br><br>
Falls fehlende Werte durch spezielle Codes wie beispielsweise <code>-999</code> dargestellt sind,  
können Sie diese mit dem Befehl <code>replace('alter_Wert', 'neuer_Wert')</code> ersetzen.  
<br><br>
Dieser Befehl ersetzt alle Vorkommen des alten Werts durch den neuen Wert im gesamten DataFrame.
</div>


In [17]:
# Ersetzen fehlender Werte (NaN) in der Spalte "O2" durch den Mittelwert der Spalte
df['O2'] = df['O2'].fillna(df['O2'].mean())

# Kontrolle: Ausgabe der Anzahl fehlender Werte pro Spalte
df.isnull().sum()

Messstelle     0
GW-Nummer      0
Datum          0
RASTERVALU     0
HYDROGEOL3    56
O2             0
NO3            0
dtype: int64

### 2.2.2 Ersetzen von NaN-Werten in kategorischen Spalten

In kategorischen Spalten werden fehlende Werte (`NaN`) typischerweise durch den am häufigsten vorkommenden Wert ersetzt,  
den sogenannten **Modus**.

Der Modus ist der Wert, der in der Spalte am häufigsten vorkommt und stellt damit eine sinnvolle Schätzung für fehlende Werte dar.


In [18]:
# Bestimmung des Modus (häufigster Wert) der Spalte "HYDROGEOL3"
hy_mode = df['HYDROGEOL3'].mode()

# Ausgabe der Häufigkeitsverteilung der Werte in der Spalte "HYDROGEOL3"
print("Häufigkeitsverteilung von 'HYDROGEOL3':")
print(df['HYDROGEOL3'].value_counts())

# Ausgabe des Modus
print("\nModus der Spalte 'HYDROGEOL3':", hy_mode.iloc[0])


Häufigkeitsverteilung von 'HYDROGEOL3':
HYDROGEOL3
Quartäre Kiese und Sande (GWL)         11955
Paläozoikum, Kristallin (GWG)             50
Tertiär im Oberrheingraben (GWG)          37
Oberjura (Raurasische Fazies) (GWL)       35
Oberer Muschelkalk (GWL)                  26
Unterjura und Mitteljura (GWG)            19
Name: count, dtype: int64

Modus der Spalte 'HYDROGEOL3': Quartäre Kiese und Sande (GWL)


In [19]:
# Ersetzen fehlender Werte in "HYDROGEOL3" durch den Modus der Spalte
df['Hydrogeologie'] = df['HYDROGEOL3'].fillna(hy_mode.iloc[0])
# Entfernen der alten Spalte "HYDROGEOL3" aus dem Datensatz
df.drop(columns='HYDROGEOL3', inplace=True)

#### Kontrolle

In [20]:
# Kontrolle: Ausgabe der Anzahl fehlender Werte pro Spalte
df.isnull().sum()

Messstelle       0
GW-Nummer        0
Datum            0
RASTERVALU       0
O2               0
NO3              0
Hydrogeologie    0
dtype: int64

## 2.3 Feature Aggregation

Wir möchten die Daten **pro Messstelle** aggregieren:

- Numerische Merkmale (**NO3**, **O2**) werden durch den **Mittelwert** je Messstelle aggregiert.
- Kategorische Merkmale (z.B. **Hydrogeologie**) werden durch den **Modus** (häufigster Wert) je Messstelle zusammengefasst.

Zur Umsetzung verwenden wir:

- Die Funktion **`groupby()`** von Pandas, um Daten nach Messstellen zu gruppieren.
- Die Funktion **`agg()`**, um verschiedene Aggregationsmethoden gezielt auf unterschiedliche Spalten anzuwenden.

Nach der Aggregation:
- Wird der Index mit **`reset_index()`** zurückgesetzt, damit die Messstellen wieder normale Spalten sind.
- Anschließend werden die getrennt aggregierten numerischen und kategorialen Daten wieder **zusammengeführt**.



In [21]:
# Aggregation der numerischen Spalten ("NO3", "O2") mit dem Mittelwert
# und der kategorialen Spalten ("Hydrogeologie", "RASTERVALU") mit dem Modus
GWM = df.groupby('Messstelle').agg({
    'NO3': 'mean',
    'O2': 'mean',
    'Hydrogeologie': pd.Series.mode,
    'RASTERVALU': pd.Series.mode
})

# Zurücksetzen des Index, damit "Messstelle" wieder eine normale Spalte ist
GWM.reset_index(inplace=True)

# Umbenennen der Spaltennamen für die kategorialen Merkmale (optional, aber für Klarheit empfohlen)
GWM.rename(columns={
    'Hydrogeologie': 'Modus_Hydrogeologie',
    'RASTERVALU': 'Modus_RASTERVALU'
}, inplace=True)

# Ausgabe des aggregierten DataFrames
GWM


,Messstelle,NO3,O2,Modus_Hydrogeologie,Modus_RASTERVALU
0,"BBR 1 Firma Schultis, Riegel",6.259259,1.315473,Quartäre Kiese und Sande (GWL),121
1,"BBR 2 Firma Thieme, Teningen",1.452941,0.823723,Quartäre Kiese und Sande (GWL),121
2,"BBR 2 Kronenwiese Firma Burda Werk 1, Offenburg",5.600000,1.984615,Quartäre Kiese und Sande (GWL),121
3,"BBR 3186 im Garten der alten Schule, Langhurst",6.968750,0.730346,Quartäre Kiese und Sande (GWL),112
4,"BBR 998 A Kehlerstraße, Neuried-Auenheim",56.164286,1.116071,Quartäre Kiese und Sande (GWL),112
...,...,...,...,...,...
502,"TB Winzergenossenschaft, Sasbach am Kaiserstuhl",23.231034,2.691992,Quartäre Kiese und Sande (GWL),112
503,"TB Wäldeleacker Feldkirch, Hartheim am Rhein",84.274074,7.785307,Quartäre Kiese und Sande (GWL),211
504,"TB1 Glockenackern, Buggingen",59.056667,7.399035,Quartäre Kiese und Sande (GWL),211
505,"TB1 Weingartenhöfe, Forchheim",69.444000,4.908000,Quartäre Kiese und Sande (GWL),211


## 2.4 Train/Test Split

Das Aufteilen des Datensatzes erfolgt mit der Funktion  
<span style="color:blue"><b><a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">train_test_split()</a></b></span>  
aus dem Paket **scikit-learn**.

Mit folgenden Parametern lässt sich das Split-Verhalten steuern:
- **`test_size`**: Legt das Verhältnis zwischen Trainings- und Testdaten fest.
- **`random_state`**: Sorgt für Reproduzierbarkeit, indem die Zufallsauswahl kontrolliert wird.

### Warum wird ein Train/Test Split durchgeführt?

Beim maschinellen Lernen ist es wichtig, die **Generalisierungsfähigkeit** eines Modells zu überprüfen.  
Das bedeutet: Wir wollen wissen, wie gut das Modell auf **neue, unbekannte Daten** reagiert – nicht nur auf die Daten, mit denen es trainiert wurde.

Daher teilen wir den Datensatz auf:
- **Trainingsdaten** (`train set`): Diese Daten werden genutzt, um das Modell zu trainieren.
- **Testdaten** (`test set`): Diese Daten bleiben während des Trainings unberührt und werden erst danach verwendet, um die Modellleistung objektiv zu bewerten.

<div style="background-color: #fff3cd; border-left: 6px solid #ffa502; padding: 10px; margin-top: 15px;">
<b>Hinweis:</b><br><br>
Ziel des Train/Test-Splits ist es, <b>Überanpassung (Overfitting)</b> zu vermeiden und eine <b>realistische Einschätzung</b> der Modellqualität zu ermöglichen.
</div>

Eine typische Aufteilung ist:
- 70–80 % Trainingsdaten
- 20–30 % Testdaten

Je nach Anwendungsfall kann das Verhältnis angepasst werden.


In [22]:
from sklearn.model_selection import train_test_split

# Aufteilen des GWM-Datensatzes in Training- und Testset
# test_size=0.8 bedeutet: 80 % der Daten werden als Testdaten verwendet
train, test = train_test_split(GWM, test_size=0.8, random_state=43)

# Ausgabe der Größe der Trainings- und Testdaten
print('Größe der Trainingsdaten:', train.shape)
print('Größe der Testdaten:', test.shape)


Größe der Trainingsdaten: (101, 5)
Größe der Testdaten: (406, 5)


## 2.5 Skalieren

Das Skalieren von Daten vor der Verwendung in einem Machine-Learning-Modell hilft dabei:
- die Leistung des Modells zu verbessern,
- die Berechnungskosten zu reduzieren,
- und Verzerrungen durch unterschiedlich große Wertebereiche zu vermeiden.

Es gibt verschiedene Skalierungsmethoden, unter anderem:
- **Min-Max-Skalierung** (Transformation auf einen festen Bereich, z. B. [0,1]),
- **Standardisierung** (Zentrierung auf Mittelwert 0 und Standardabweichung 1),
- **Logarithmische Skalierung** (für stark verzerrte Datenverteilungen).

> Die beste Methode hängt von den Eigenschaften der Daten und dem eingesetzten Modell ab.

### 2.5.1 Normieren

Normieren ist ein Verfahren, bei dem Daten auf eine gemeinsame **Einheitsnorm** gebracht werden,  
indem jeder Wert durch die **Länge des Vektors** (z.B. euklidische Norm) dividiert wird.

- Dadurch bleibt die Richtung der Daten erhalten,
- jedoch spielt die absolute Skala der Werte keine Rolle mehr.

Normierung wird häufig in Machine-Learning-Algorithmen eingesetzt, bei denen die **Datenrichtung** wichtiger ist als ihre tatsächliche Größe (z.B. bei **k-nächste Nachbarn**, **Clustering** oder **Cosinus-Ähnlichkeitsbere**


#### 2.5.1.1 Händisches Skalieren mit Hilfe der Formel:
$X_{norm} = \frac{X - X_{min}}{X_{max} - X_{min}}$

In [23]:
# Minimum und Range der Spalte "NO3" im Trainingsdatensatz berechnen
X_min = train['NO3'].min()
X_range = train['NO3'].max() - train['NO3'].min()

# Normalisieren der Spalte "NO3" im Trainingsdatensatz (Min-Max-Skalierung)
train['NO3_norm'] = (train['NO3'] - X_min) / X_range

# Normalisieren der Spalte "NO3" im Testdatensatz
# Achtung: Normierung basiert auf den Trainingsdaten, um Datenleckage zu vermeiden!
test['NO3_norm'] = (test['NO3'] - X_min) / X_range

# Ausgabe einer statistischen Zusammenfassung der normalisierten "NO3"-Spalte im Trainingsdatensatz
print(train['NO3_norm'].describe()[['min', 'max', 'mean']])

min     0.000000
max     1.000000
mean    0.204514
Name: NO3_norm, dtype: float64


#### 2.5.1.1 Verwendung der vorhandenen Funktion "MinMaxScaler" aus sklearn

Anstatt eine Min-Max-Skalierung manuell durchzuführen, kann die vorhandene Funktion **`MinMaxScaler`** aus **scikit-learn** verwendet werden.

Dabei folgt das Vorgehen einem festen Ablauf:
- Zuerst wird ein eigenes **Scaler-Objekt** erstellt.
- Dieses Objekt wird anschließend an den Trainingsdaten **gefitttet** (`fit()`).
- Danach können die Trainings- und Testdaten basierend auf diesem Fit **transformiert** werden (`transform()`).

<div style="background-color: #e7f3fe; border-left: 6px solid #2196F3; padding: 10px; margin-top: 15px;">
<b>Wichtig:</b><br><br>
Der Scaler wird ausschließlich an den Trainingsdaten gefittet, um Datenleckage zu vermeiden.  
Die Testdaten werden nur transformiert, niemals beim Fit berücksichtigt.
</div>



In [24]:
# --- Codeabschnitt: bitte beachten Sie die Funktion dieses Blocks ---
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler-Objekt erstellen und auf Trainingsdatensatz anpassen
scaler = MinMaxScaler()
scaler.fit(train[['NO3']])

# Spalte NO3 im Trainings- und Testdatensatz normalisieren und als neue Spalte hinzufügen
train['NO3_norm'] = scaler.transform(train[['NO3']])
test['NO3_norm'] = scaler.transform(test[['NO3']])

print(train['NO3_norm'].describe()[['min', 'max', 'mean']])

min     0.000000
max     1.000000
mean    0.204514
Name: NO3_norm, dtype: float64


### 2.5.2 Standardisieren

Die **Standardisierung** ist ein Verfahren, bei dem Daten so transformiert werden,  
dass sie eine **Standardnormalverteilung** mit einem Mittelwert von **0** und einer Standardabweichung von **1** erhalten.

Die Standardisierung erfolgt durch:
- Subtraktion des Mittelwerts der Daten,
- Division durch die Standardabweichung der Daten.

Dadurch werden die Daten **zentriert** (auf Mittelwert 0) und **skaliert** (auf Standardabweichung 1).

Die Standardisierung kann helfen:
- die Auswirkungen von Ausreißern zu reduzieren,
- und die Modellleistung zu verbessern, insbesondere bei **linearen Modellen** wie der **linearen Regression**.


#### 2.5.2.1 Händisches Skalieren mit Hilfe der Formel

Die Standardisierung eines Werts \( X \) erfolgt über folgende Formel:


$X_{\text{std}} = \frac{X - \text{mean}(X)}{\text{std}(X)}$


In [25]:
# Berechnen von Mittelwert und Standardabweichung der NO3-Spalte im Trainingsdatensatz
X_mean = train['NO3'].mean()
X_std = train['NO3'].std()

# Erstellen einer neuen Spalte im Trainings- und Testdatensatz durch Standardisierung der NO3-Spalte,
# basierend auf dem Mittelwert und der Standardabweichung des Trainingsdatensatzes, 
# um Datenleckage zu vermeiden
train['NO3_std'] = (train['NO3'] - X_mean) / X_std
test['NO3_std'] = (test['NO3'] - X_mean) / X_std

# Ausgabe des Mittelwerts und der Standardabweichung der standardisierten Spalte NO3_std im Trainingsdatensatz
print('mean(X): ', train['NO3_std'].mean())
print('std(X): ', train['NO3_std'].std())

mean(X):  5.27630744376312e-17
std(X):  0.9999999999999999


#### 2.5.2.2 Vorhandene Funktion von sklearn "StandardScaler"

In [26]:
# Berechnen von Mittelwert und Standardabweichung der "NO3"-Spalte im Trainingsdatensatz
X_mean = train['NO3'].mean()
X_std = train['NO3'].std()

# Standardisierung der "NO3"-Spalte im Trainingsdatensatz
train['NO3_std'] = (train['NO3'] - X_mean) / X_std

# Standardisierung der "NO3"-Spalte im Testdatensatz
# Hinweis: Mittelwert und Standardabweichung stammen ausschließlich aus den Trainingsdaten (Datenleckage vermeiden)
test['NO3_std'] = (test['NO3'] - X_mean) / X_std

# Ausgabe des Mittelwerts und der Standardabweichung der standardisierten Spalte im Trainingsdatensatz
print('Mittelwert von NO3_std (Training):', round(train['NO3_std'].mean(), 4))
print('Standardabweichung von NO3_std (Training):', round(train['NO3_std'].std(), 4))


Mittelwert von NO3_std (Training): 0.0
Standardabweichung von NO3_std (Training): 1.0


## 2.6 Encodieren

Encodieren bezeichnet die Konvertierung von Daten in eine geeignete Darstellung,  
die von einer Maschine oder einem Computer verarbeitet werden kann.

Dies umfasst typischerweise die Umwandlung von Daten in:
- numerische,
- binäre,
- oder kategoriale Darstellungen,

je nachdem, welche Anforderungen das zu trainierende Modell stellt.

*Die Wahl der richtigen Encoding-Methode kann einen erheblichen Einfluss auf die Leistung eines Machine-Learning-Modells haben.*


### 2.6.1 Binäres Encodieren

Beim binären Encodieren wird jede Kategorie in einen einfachen Zahlenwert umgewandelt.

**Beispiel: CORINE-Landnutzungscodes**

Der dreistellige CORINE-Code stellt eine hierarchische Beschreibung der Landnutzung dar:
- **1XX** = Artificial Surfaces
- **2XX** = Agricultural Areas
- **3XX** = Forest and Seminatural Areas
- **4XX** = Wetlands
- **5XX** = Water Bodies

Dabei steht die erste Ziffer für die oberste Kategorie.


Im Folgenden erfolgt ein **binäres Encoding** auf Basis der **ersten Ziffer**:

- Zunächst wird eine neue Spalte erstellt, die nur die **erste Ziffer** des dreistelligen Codes enthält.
- Anschließend erfolgt die Umwandlung in eine **binäre Darstellung**,  
  abhängig davon, zu welcher Hauptkategorie der ursprüngliche Code gehört.


In [27]:
# Binäres Encoding der Landnutzung: Artificial Surfaces (1XX-Codes) vs. andere Klassen

datasets = [train, test]

for dataset in datasets:
    # Extrahiere die erste Ziffer aus der Spalte "Modus_RASTERVALU"
    dataset['Corine'] = dataset['Modus_RASTERVALU'].apply(lambda x: int(str(x)[0]))
    
    # Erzeuge die binäre Spalte "Artificial_Surface"
    # 1, wenn erste Ziffer = 1 (Artificial Surfaces), sonst 0
    dataset['Artificial_Surface'] = dataset['Corine'].apply(lambda x: 1 if x == 1 else 0)
    
    # Entferne die ursprüngliche Spalte "Modus_RASTERVALU", da sie nicht mehr benötigt wird
    dataset.drop(columns='Modus_RASTERVALU', inplace=True)


### 2.6.2 One-Hot-Encoding

One-Hot-Encoding ist ein Verfahren zur Darstellung von **kategorischen** oder **Nominaldaten** (Daten ohne natürliche Rangfolge) als numerische Vektoren.

Dabei wird:
- jeder eindeutigen Kategorie eine eigene neue Spalte zugewiesen,
- und innerhalb dieser Spalte wird für jede Beobachtung ein **Binärwert** gesetzt (1 oder 0).

Das bedeutet:
- Eine "1" zeigt an, dass eine Beobachtung zu dieser Kategorie gehört.
- Eine "0" zeigt an, dass sie nicht zu dieser Kategorie gehört.

One-Hot-Encoding wird insbesondere für **nominale** Merkmale eingesetzt,  
um sie in eine Form zu bringen, die von maschinellen Lernmodellen verarbeitet werden kann, ohne eine künstliche Rangordnung zu implizieren.


Beispiel für One-Hot Encoding:

In [28]:
# --- Codeabschnitt: One-Hot-Encoding einer kategorialen Spalte ---

from sklearn.preprocessing import OneHotEncoder

# Erstellen eines OneHotEncoder-Objekts (Ausgabe als dichte Matrix, nicht als Sparse-Matrix)
encoder = OneHotEncoder(sparse_output=False)

# Anpassen (Fit) des Encoders an die "Corine"-Spalte des Trainingsdatensatzes
encoder.fit(train[['Corine']])

# Transformation der "Corine"-Spalte im Testdatensatz (nur transformieren, nicht nochmal fitten!)
OHE_test = encoder.transform(test[['Corine']])

# Konvertieren der One-Hot-kodierten Ergebnisse in ein DataFrame
OHE_test = pd.DataFrame(OHE_test, columns=encoder.get_feature_names_out(['Corine']))

# Ausgabe der One-Hot-kodierten Testdaten
OHE_test


,Corine_1,Corine_2,Corine_3,Corine_5
0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
401,1.0,0.0,0.0,0.0
402,0.0,0.0,1.0,0.0
403,0.0,1.0,0.0,0.0
404,1.0,0.0,0.0,0.0


### 2.6.3 Target Encoding

Beim **Target Encoding** wird der Mittelwert der Zielvariable (Target) für jede Kategorie berechnet und anschließend als numerischer Wert für diese Kategorie verwendet.

Das Verfahren läuft typischerweise so ab:
- Für jede Ausprägung einer kategorialen Variable wird der Durchschnittswert der Zielvariable ermittelt.
- Dieser Durchschnitt ersetzt die ursprüngliche Kategorie.

<div style="background-color: #fff3cd; border-left: 6px solid #ffa502; padding: 10px; margin-top: 15px;">
<b>Wichtig:</b><br><br>
Das Target Encoding muss <b>nach</b> dem Train/Test-Split durchgeführt werden,  
damit keine Informationen aus den Testdaten in das Training einfließen (Vermeidung von Data Leakage).
</div>


In [29]:
# --- Codeabschnitt: Target Encoding einer kategorialen Variable ---

from category_encoders.target_encoder import TargetEncoder

# Erstellen eines TargetEncoder-Objekts
encoder = TargetEncoder()

# Fit des Encoders auf den Trainingsdaten und Transformation der "Modus_Hydrogeologie"-Spalte
train['Target_Hy'] = encoder.fit_transform(train['Modus_Hydrogeologie'], train['NO3'])

# Transformation der "Modus_Hydrogeologie"-Spalte im Testdatensatz (ohne neues Fitting!)
test['Target_Hy'] = encoder.transform(test['Modus_Hydrogeologie'])

# Ausgabe der ersten Zeilen der Trainingsdaten zur Kontrolle
train.head()


,Messstelle,NO3,O2,Modus_Hydrogeologie,NO3_norm,NO3_std,Corine,Artificial_Surface,Target_Hy
49,"BR 7 Hofgut Freudenberg, Heddesheim",0.366000,0.394000,Quartäre Kiese und Sande (GWL),0.001255,-0.843072,2,0,21.923784
298,"GWM E40 ZWK Kurpfalz, Schwetzingen",15.027586,5.883984,Quartäre Kiese und Sande (GWL),0.140704,-0.264668,1,1,21.923784
437,TB 2 Transportbeton Freiburg,16.500000,6.939286,Quartäre Kiese und Sande (GWL),0.154709,-0.206581,1,1,21.923784
441,"TB 3 Firma Hansa Heemann AG, Bruchsal",0.287037,0.546296,Quartäre Kiese und Sande (GWL),0.000504,-0.846187,1,1,21.923784
382,"GWM Tief, Heddesheim",0.250000,0.347619,Quartäre Kiese und Sande (GWL),0.000151,-0.847648,2,0,21.923784


## 2.7 Speichern der Ergebnisse

Um einen Datensatz zu speichern, ist es sinnvoll, ihn als **CSV-Datei** zu exportieren.

Das Exportieren eines DataFrames erfolgt mit dem Befehl  
<span style="color:blue"><b><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html">.to_csv()</a></b></span>  
aus der Pandas-Bibliothek.

Damit kann der bearbeitete Datensatz dauerhaft gespeichert und später einfach wieder eingelesen werden.


In [30]:

# Speichern des Trainingsdatensatzes als CSV-Datei im Ordner "Termin_1"
# Hinweis: Ohne Angabe von "index=False" wird der Index als zusätzliche Spalte mitgespeichert
train.to_csv('res.csv', index=False)


# 3. Übung: Eigene Datenaufbereitung

Bereite den Datensatz **"Nitratmessungen_aufgabe.csv"** eigenständig auf:

### Aufgaben:

1. Ersetze alle Werte unterhalb der Nachweisgrenze (NWG).
2. Ersetze alle fehlenden Werte (`NULL`-Values).
3. Aggregiere die Daten für jede Messstelle.
4. Splitte den Datensatz im Verhältnis 85 % Training : 15 % Test.
5. Standardisiere die numerischen Variablen.
6. Führe ein binäres Encoding der Landnutzung durch:
   - **Landwirtschaft = 2XX** ➔ `1`
   - **Alle anderen Klassen** ➔ `0`
7. Target-Encode die Hydrogeologie basierend auf der Nitratkonzentration.
8. Speichere deine Ergebnisse.
9. Fertig!

---

<div style="background-color: #fff3cd; border-left: 6px solid #ffa502; padding: 10px; margin-top: 15px;">
<b>Tipps:</b><br><br>
- Schaue dir die Features zur <b>Sauerstoff-Konzentration</b> und <b>Hydrogeologie</b> genau an.<br>
- Fehlende Werte sind hier nicht immer als klassische <code>NULL</code> oder <code>NaN</code> dargestellt.<br>
- Nutze zur Analyse:<br>
  - <code>.describe()</code> → Statistische Übersicht<br>
  - <code>.unique()</code> → Alle vorkommenden Werte
</div>
